In [1]:
import pandas as pd 
import numpy as np 

In [3]:
df = pd.read_csv("/Users/vaishnavimocherla/Desktop/Rh/Data/worldcities.csv")

In [4]:
df.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6870,139.7495,Japan,JP,JPN,Tōkyō,primary,37785000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48059 entries, 0 to 48058
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        48059 non-null  object 
 1   city_ascii  48057 non-null  object 
 2   lat         48059 non-null  float64
 3   lng         48059 non-null  float64
 4   country     48059 non-null  object 
 5   iso2        48026 non-null  object 
 6   iso3        48059 non-null  object 
 7   admin_name  47858 non-null  object 
 8   capital     15138 non-null  object 
 9   population  47808 non-null  float64
 10  id          48059 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 4.0+ MB


In [10]:
unique=df['country'].nunique()
unique

242

In [12]:
print(df.shape)

(48059, 11)


In [18]:
df.isna().sum()

city          0
city_ascii    0
lat           0
lng           0
country       0
iso2          0
iso3          0
admin_name    0
capital       0
population    0
id            0
dtype: int64

In [ ]:
df.dropna(subset=['population', 'capital', 'admin_name'], inplace=True)
df['iso2'].fillna('N/A', inplace=True)


/var/folders/3k/3lbygck964ldccpncrf48vsw0000gn/T/ipykernel_24583/3067239811.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['iso2'].fillna('N/A', inplace=True)


In [17]:
df['city_ascii'].fillna("N/A", inplace = True)

In [20]:
df.shape

(14880, 11)

In [21]:
df.nunique()

city          14310
city_ascii    14274
lat           12731
lng           13395
country         201
iso2            201
iso3            201
admin_name     3586
capital           3
population    13499
id            14880
dtype: int64

In [29]:
pop_by_country = df.groupby(['country']).agg({
    'population': "mean",
    'id' :'count'
}

)

top10 = pop_by_country.sort_values(by='population', ascending=False).head(10)
print(top10)

                  population   id
country                          
United Kingdom  1.126200e+07    1
India           3.856634e+06   38
Kuwait          2.989000e+06    1
Bangladesh      2.488296e+06   10
Australia       2.103341e+06    8
South Africa    1.897159e+06    9
Japan           1.858691e+06   49
Puerto Rico     1.809800e+06    1
China           1.353010e+06  704
Korea, South    1.344218e+06   34


In [32]:
pop_by_city = df.groupby(['city']).agg({
    'population': "mean",
    'country' : lambda x : x.mode().iloc[0] if not x.mode().empty else None
}

)
pop_by_city
top10 = pop_by_city.sort_values(by='population', ascending=False).head(10)
print(top10)

             population       country
city                                 
Tokyo        37785000.0         Japan
Jakarta      33756000.0     Indonesia
Delhi        32226000.0         India
Guangzhou    26940000.0         China
Mumbai       24973000.0         India
Manila       24922000.0   Philippines
Shanghai     24073000.0         China
São Paulo    23086000.0        Brazil
Seoul        23016000.0  Korea, South
Mexico City  21804000.0        Mexico


In [35]:
pop_by_country = df.groupby('country').agg({
    'population': 'mean',           # avg city population
    'lat': 'mean',                  # centroid latitude
    'lng': 'mean',                  # centroid longitude
})
top10_ll = pop_by_country.sort_values(by = 'population', ascending = False).head(10)

In [36]:
top10_ll

,population,lat,lng
country,,,
United Kingdom,1.126200e+07,51.507200,-0.127500
India,3.856634e+06,21.986800,81.373782
Kuwait,2.989000e+06,29.369700,47.978300
Bangladesh,2.488296e+06,24.148850,90.072320
Australia,2.103341e+06,-32.080475,141.367987
South Africa,1.897159e+06,-28.440278,26.505911
Japan,1.858691e+06,35.289335,135.898082
Puerto Rico,1.809800e+06,18.398500,-66.061000
China,1.353010e+06,34.008437,111.050488


In [39]:
idx_n = df.groupby('country')['lat'].idxmax()
idx_s = df.groupby('country')['lat'].idxmin()
idx_e = df.groupby('country')['lng'].idxmax()
idx_w = df.groupby('country')['lng'].idxmin()

extremes = pd.DataFrame({
    'north_city': df.loc[idx_n, 'city'].values,
    'south_city': df.loc[idx_s, 'city'].values,
    'east_city' : df.loc[idx_e, 'city'].values,
    'west_city' : df.loc[idx_w, 'city'].values
}, index=df.loc[idx_n, 'country'].values)

extremes.head(10)

,north_city,south_city,east_city,west_city
Afghanistan,Qarqīn,Zaranj,Asadābād,Kuhsān
Albania,Kukës,Gjirokastër,Korçë,Durrës
Algeria,Annaba,In Guezzam,Djanet,Tindouf
Andorra,Canillo,Sant Julià de Lòria,Canillo,Sant Julià de Lòria
Angola,Cabinda,Ondjiva,Dundo,Moçâmedes
Antigua and Barbuda,Saint John’s,Saint John’s,Saint John’s,Saint John’s
Argentina,La Quiaca,Ushuaia,Bernardo de Irigoyen,El Calafate
Armenia,Ijevan,Kapan,Kapan,Gyumri
Australia,Darwin,Hobart,Brisbane,Perth
Austria,Zwettl,Klagenfurt,Gänserndorf,Dornbirn


In [41]:
by_country = (
    df.groupby('country')
      .agg(
          avg_pop=('population', 'mean'),
          n_cities=('city', 'nunique'),
          lat_mean=('lng', 'mean'),
          lon_mean=('lng', 'mean'),
          lat_median=('lat', 'median'),
          lon_median=('lng', 'median'),
          lat_min=('lat', 'min'),
          lat_max=('lat', 'max'),
          lon_min=('lng', 'min'),
          lon_max=('lng', 'max'),
          lat_std=('lat', 'std'),
          lon_std=('lng', 'std'),
      )
)
by_country

,avg_pop,n_cities,lat_mean,lon_mean,lat_median,lon_median,lat_min,lat_max,lon_min,lon_max,lat_std,lon_std
country,,,,,,,,,,,,
Afghanistan,215930.000000,50,67.149350,67.149350,34.65915,67.92500,30.9600,37.4128,61.1667,71.1528,1.664256,2.744275
Albania,88781.000000,12,19.936225,19.936225,41.21220,19.88415,40.0758,42.0833,19.4458,20.7667,0.658508,0.436772
Algeria,166776.448276,58,3.440509,3.440509,35.40000,3.56955,19.5686,36.9000,-8.1286,9.4853,4.150303,3.439016
Andorra,11870.285714,7,1.537457,1.537457,42.53610,1.53310,42.4650,42.5667,1.4911,1.6000,0.035643,0.040991
Angola,864251.555556,18,15.543856,15.543856,-10.42765,14.98335,-17.0667,-5.5600,12.1508,20.8351,3.296764,2.694076
...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,434285.201754,114,106.441549,106.441549,13.47195,106.33640,8.6931,22.8333,103.0167,109.2950,4.888208,1.325732
Wallis and Futuna,675.500000,2,-177.165950,-177.165950,-13.78790,-177.16595,-14.2933,-13.2825,-178.1583,-176.1736,0.714744,1.403395
Yemen,168017.230769,39,45.309490,45.309490,14.51610,44.73140,12.6500,16.9400,42.9511,54.0167,1.184301,2.483839


In [42]:
df.to_csv('worldnew.csv')